In [2]:
using ITensors, ITensorMPS
import Pkg
Pkg.activate(".")
using ITensors

# Q_ij function
function Q_ij(i, j, n, sites)
    os = OpSum()
    if i == j
        coeff = 1 / (2 * sqrt(2 * i * (i - 1)))
        for c in 1:i-1
            os +=  coeff, "Z", 3*(n-1) + c      # shift to 1-index
            os += -coeff, "Z", 3*(n-1) + i
        end
    else
        si = 3*(n-1) + i
        sj = 3*(n-1) + j
        # Build ("S+", si, "Z", sj, "Z", ..., "S+", sj) sequence
        z_chain = collect(Iterators.flatten([("Z", k) for k in sj:si]))
        os += 1.0, "S+", si, z_chain..., "S+", sj
    end
    return os
end

# Q function
function Q(n)
    os = OpSum()
    os +=  1.0, "Id", 3*(n-1)+1, "Id", 3*(n-1)+2
    os +=  1.0, "Z",  3*(n-1)+1, "Id", 3*(n-1)+2
    os += -1.0, "Id", 3*(n-1)+1, "Z",  3*(n-1)+2
    os += -1.0, "Z",  3*(n-1)+1, "Z",  3*(n-1)+2
    return os
end

# Main block
let
    m = 1
    N = 24
    sites = siteinds("S=1/2", N)
    states = [isodd(n) ? "Up" : "Dn" for n in 1:N]
    psi0 = MPS(sites, states)

    # --- Mass Term ---
    H_mass = OpSum()
    for j in 2:(N - 1)
        coeff = (m / 2) * (isodd(div(j,3)) ? 1 : -1)
        H_mass += coeff, "Z", j
    end

    # --- Electric Field Term ---
    H_elec = OpSum()
    for n in 1:(N - 1)
        for i in 2:3
            for j in 1:(i - 1)
                H_elec += 2.0 * Q_ij(i, j, n, sites)
            end
        end
    end

    # --- Combine all Hamiltonian terms symbolically ---
    H_total = H_mass + H_elec

    # Convert once to MPO
    H = MPO(H_total, sites)

    # --- Run DMRG ---
    nsweeps = 5
    maxdim = [10, 20, 100, 100, 200]
    cutoff = 1.0e-10

    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff)
    println("Final energy = $energy")
end


  Activating project at `c:\Users\aradh\Desktop\Physic 697\physics_697_fall_2025_Max_Aradh_project`


ErrorException: The OpSum contains a term 2.0 S+(26,) Z(25,) Z(26,) S+(25,) that extends beyond the number of sites 24.